**<p style = "text-align: center">TRƯỜNG ĐẠI HỌC KHOA HỌC TỰ NHIÊN - ĐẠI HỌC QUỐC GIA TPHCM</p>**

# **<p style = "text-align: center">LẬP TRÌNH KHOA HỌC DỮ LIỆU</p>**
## **<p style = "text-align: center">Đồ án Cuối kì</p>**
### <p style = "text-align: center"> <b>Đề tài:</b> Xử lí, phân tích và khám phá dữ liệu về sức khỏe tinh thần của nhân viên công nghệ (Mental Health in Tech)</p>
*<p style = "text-align: center">Bộ dữ liệu này là từ một cuộc khảo sát năm 2014 đo lường thái độ đối với sức khỏe tâm thần và tần suất rối loạn sức khỏe tâm thần của những người làm việc trong lĩnh vực công nghệ.</p>*
**Sinh viên thực hiện:**

| Mã số sinh viên | Họ và tên |
| --- | --- |
| 21120576 | Trần Đình Nhật Trí |
| 21120590 | Nguyễn Thủy Uyên |
| 21120501 | Nguyễn Ngọc Gia Minh |

## Import thư viện

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_df = pd.read_csv('survey.csv')
raw_df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


## Với mỗi cột có kiểu là dữ liệu số, những giá trị trong cột đó được phân bố như thế nào?
Để trả lời câu hỏi này, trước tiên ta sẽ lọc những cột có kiểu dữ liệu số có trong `raw_df`:

In [3]:
numeric_cols = raw_df.select_dtypes(include='number')
numeric_cols

,Age
0,37
1,44
2,32
3,31
4,31
...,...
1254,26
1255,32
1256,34
1257,46


Như ta thấy, chỉ có duy nhất cột `Age` là cột có dữ liệu số.
Ta sẽ kiểm tra tổng quan giá trị trong `Age` được phân bố như thế nào:

In [4]:
# plt.figsize=(15,2)
# sns.histplot(numeric_col, x='Age')
pd.set_option('display.float_format', lambda x : '%.2f' %x)
numeric_cols.describe()

,Age
count,1259.00
mean,79428148.31
std,2818299442.98
min,-1726.00
25%,27.00
50%,31.00
75%,36.00
max,99999999999.00


Trong cột `Age` có dữ liệu bất bình thường, nguyên nhân là do: Số tuổi **lớn nhất** `Max` là rất lớn và số tuổi **bé nhất** `Min` là một số âm. Hai giá trị này là rất vô lý khi nó xuất hiện trong thuộc tính độ tuổi.

Tiếp theo ta kiểm tra tỉ lệ missing values chiếm trong cột này là bao nhiêu:

In [5]:
percent_missing = numeric_cols.isnull().sum() * 100 / len(numeric_cols)
percent_missing

Age   0.00
dtype: float64

Không có missing values nào tồn tại trong `Age`.

**Nhận xét:**
- Cột dữ liệu số trong bộ dữ liệu này là bất bình thường do xuất hiện những giá trị vô lý trong bảng dữ liệu, khiến dữ liệu bị sai lệch. Chúng được gọi là các giá trị ngoại lai `outliers`.
- Độ tuổi làm việc trong môi trường công nghệ thường nằm trong phạm vi **18 đến 75 tuổi**. Do đó trong quá trình tìm hiểu dữ liệu, ta sẽ cần phải loại bỏ những `outliers` này.

## Với các cột dữ liệu kiểu categorical, ta xem xét sự phân bố

In [6]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

cate_cols_df =  raw_df.select_dtypes(exclude = 'number')
cate_cols_df

,Timestamp,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,Female,United States,IL,NaN,No,Yes,Often,6-25,No,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,Male,Canada,NaN,NaN,No,No,Rarely,6-25,No,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,Male,United States,TX,NaN,No,No,Never,100-500,Yes,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,No,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,Male,United States,IL,No,Yes,Yes,Often,26-100,Yes,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,No,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,f,United States,NC,No,No,No,NaN,100-500,Yes,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


### Kiểm tra tỉ lệ giá trị bị thiếu, số các giá trị khác biệt, sự phân bố dữ liệu ở các cột kiểu categorical
Nhận xét: Ta nhận thấy ở cột **Gender** có một số giá trị ngoại lai (bất thường như *mail, m, msle, csi-male, non-binary, All ...*). Đối với các cột còn lại ngoại trừ cột **comments** các giá trị trông có vẻ không có gì đáng ngại

In [7]:
def missing_ratios(col):
    return col.isnull().sum() * 100 / len(col)


cate_cols = cate_cols_df.agg(['nunique', lambda x: x.value_counts().to_dict(), missing_ratios])
cate_cols.index = ['num_diff_vals', 'distribution', 'missing_ratio']
cate_cols

,Timestamp,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
num_diff_vals,1246,49,48,45,2,2,2,4,6,2,...,5,3,3,3,3,3,3,3,2,160
distribution,"{'2014-08-27 12:44:51': 2, '2014-08-27 14:22:4...","{'Male': 615, 'male': 206, 'Female': 121, 'M':...","{'United States': 751, 'United Kingdom': 185, ...","{'CA': 138, 'WA': 70, 'NY': 57, 'TN': 45, 'TX'...","{'No': 1095, 'Yes': 146}","{'No': 767, 'Yes': 492}","{'Yes': 637, 'No': 622}","{'Sometimes': 465, 'Never': 213, 'Rarely': 173...","{'6-25': 290, '26-100': 289, 'More than 1000':...","{'No': 883, 'Yes': 376}",...,"{'Don't know': 563, 'Somewhat easy': 266, 'Ver...","{'No': 490, 'Maybe': 477, 'Yes': 292}","{'No': 925, 'Maybe': 273, 'Yes': 61}","{'Some of them': 774, 'No': 260, 'Yes': 225}","{'Yes': 516, 'No': 393, 'Some of them': 350}","{'No': 1008, 'Maybe': 207, 'Yes': 44}","{'Maybe': 557, 'No': 500, 'Yes': 202}","{'Don't know': 576, 'Yes': 343, 'No': 340}","{'No': 1075, 'Yes': 184}","{'* Small family business - YMMV.': 5, 'I'm no..."
missing_ratio,0.00,0.00,0.00,40.91,1.43,0.00,0.00,20.97,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,86.97


In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
raw_df.describe(include = 'object')

,Timestamp,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
count,1259,1259,1259,744,1241,1259,1259,995,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,1259,164
unique,1246,49,48,45,2,2,2,4,6,2,2,3,3,3,3,3,5,3,3,3,3,3,3,3,2,160
top,2014-08-27 12:44:51,Male,United States,CA,No,No,Yes,Sometimes,6-25,No,Yes,Yes,No,No,No,Don't know,Don't know,No,No,Some of them,Yes,No,Maybe,Don't know,No,* Small family business - YMMV.
freq,2,615,751,138,1095,767,637,465,290,883,1031,477,501,842,646,819,563,490,925,774,516,1008,557,576,1075,5
